<a href="https://colab.research.google.com/github/mar7i4ka/AI-Agent-Models/blob/main/RL_MAZE_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random, time, sys, math
import matplotlib.pyplot as plt

# 1. PARAMETERS & MAZE INITIALISATION
W, H = 6, 5
START = (0, 0)
GOAL  = (4, 5)
ACTIONS = [(-1,0), (1,0), (0,-1), (0,1)]  # L,R,U,D
ACTION_NAMES = ["←", "→", "↑", "↓"]

# Probabilities: 0.8 success, 0.1 stay, 0.1 bounce back
P_SUCCESS, P_STAY, P_BOUNCE = 0.8, 0.1, 0.1
GAMMA = 0.95                  # discount
STEP_COST = -1.0
GOAL_REWARD = 100.0
TOL = 1e-3                    # Bellman tolerance

HIDDEN_OBS = {(1,1), (2,2), (3,3)}

# 2. MDP DATA STRUCTURES
S = [(r, c) for r in range(H) for c in range(W)]
S_idx = {s:i for i,s in enumerate(S)}
N = len(S)

# Transition tensor T[s,a,s'] – will be recomputed each time a new obstacle is found
T = np.zeros((N, len(ACTIONS), N))
R = np.full((N, len(ACTIONS)), STEP_COST)
R[S_idx[GOAL], :] = 0.0

known_blocked = set()

def inside(r, c):
    return 0 <= r < H and 0 <= c < W

# 3. DYNAMIC REPLANNING ROUTINES

def rebuild_transition():
    """Re‑create T after the set `known_blocked` changes."""
    global T
    T = np.zeros_like(T)
    for s,(r,c) in enumerate(S):
        if (r,c)==GOAL or (r,c) in known_blocked:
            T[s,:,s] = 1.0
            continue
        for a,(dr,dc) in enumerate(ACTIONS):
            tr, tc = r+dr, c+dc
            # intended cell reachable?
            if not inside(tr,tc) or (tr,tc) in known_blocked:
                tr, tc = r, c  # treat as wall – stay put
            # bounce cell (opposite dir)
            br, bc = r-dr, c-dc
            if not inside(br,bc) or (br,bc) in known_blocked:
                br, bc = r, c
            T[s,a,S_idx[(tr,tc)]] += P_SUCCESS
            T[s,a,S_idx[(r,c)]]    += P_STAY
            T[s,a,S_idx[(br,bc)]]  += P_BOUNCE

rebuild_transition()


def value_iteration():
    """Return V* array and greedy policy π* (list of action indices)."""
    V = np.zeros(N)
    while True:
        Δ = 0.0
        for s in range(N):
            q = R[s] + GAMMA * T[s].dot(V)
            v_new = q.max()
            Δ = max(Δ, abs(v_new - V[s]))
            V[s] = v_new
        if Δ < TOL * (1 - GAMMA) / GAMMA:
            break
    π = np.argmax(R + GAMMA * T @ V, axis=1)
    return V, π

# 4. EXECUTION LOOP

def simulate():
    pos = START
    trajectory = [pos]
    steps = 0
    while pos != GOAL:
        V, π = value_iteration()
        a = π[S_idx[pos]]
        intended = (pos[0]+ACTIONS[a][0], pos[1]+ACTIONS[a][1])
        rnd = random.random()
        if rnd < P_SUCCESS:
            nxt = intended
        elif rnd < P_SUCCESS+P_STAY:
            nxt = pos
        else:
            nxt = (pos[0]-ACTIONS[a][0], pos[1]-ACTIONS[a][1])
        if not inside(*nxt):
            nxt = pos
        # reveal obstacle if stepped into hidden block
        if nxt in HIDDEN_OBS:
            known_blocked.add(nxt)
            rebuild_transition()
            nxt = pos  # stay on bump
        pos = nxt
        trajectory.append(pos)
        steps += 1
        render(pos, trajectory)
    print(f"Reached goal in {steps} steps.")

# 5. VISUALISATION

def render(agent, traj):
    grid = np.full((H,W), "·", dtype=object)
    for r,c in HIDDEN_OBS:     grid[r,c] = "?"  # unseen obstacles
    for r,c in known_blocked:  grid[r,c] = "#"  # discovered walls
    ar,ac = agent
    grid[ar,ac] = "A"
    sr,sc = START; grid[sr,sc] = "S"
    gr,gc = GOAL;  grid[gr,gc] = "G"
    print("\n".join(" ".join(row) for row in grid))
    print("-"*20)
    time.sleep(0.2)

if __name__ == "__main__":
    simulate()

S A · · · ·
· ? · · · ·
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · A · · ·
· ? · · · ·
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · A · ·
· ? · · · ·
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · · A ·
· ? · · · ·
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · · · A
· ? · · · ·
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · · · A
· ? · · · ·
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · · · ·
· ? · · · A
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · · · ·
· ? · · · ·
· · ? · · A
· · · ? · ·
· · · · · G
--------------------
S · · · · ·
· ? · · · A
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · · · ·
· ? · · · A
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · · · ·
· ? · · · A
· · ? · · ·
· · · ? · ·
· · · · · G
--------------------
S · · · · ·
· ? · · · ·
· · ? · · A
· · · ? · ·
· · · · · G
--------------------
S · · · · ·
· ? · · · ·
· · 